In [0]:
%run ../utils

In [0]:
dbutils.widgets.text("adhoc_date", "")

In [0]:
adhoc_date = dbutils.widgets.get("adhoc_date")
# display(adhoc_date)

In [0]:
date = genrate_yesterdays_date(adhoc_date)
print(f"current job is running for the date: {date}")

# create dataframe on input dataset of customer transaction
trans_df = spark.read.parquet(f"/mnt/silver-layer/{date}/customer_transaction")

# create dataframe on input dataset of customer profile
customer_df = spark.read.parquet(f"/mnt/silver-layer/{date}/customer_profile")

print(trans_df.show())
print(customer_df.show())

In [0]:
join_df = trans_df.join(customer_df,"customer_id","left")
# print((join_df.show()))

window_spec =Window.partitionBy("customer_id")

df = join_df.select("customer_id","first_name","last_name","amount").withColumn("sum_amount", sum("amount").over(window_spec)).drop("amount").distinct()
print(df.show())

df.coalesce(1).write.mode("overwrite").parquet(f"/mnt/gold-layer/{date}/Total_Spending_Per_Customer")




In [0]:
category_spend_df = join_df.select("category","amount").groupby("category").agg(sum("amount").cast(DecimalType(18,2)).alias("total_spend"))

category_spend_df = category_spend_df.orderBy(col("total_spend").desc())
print(category_spend_df.show())

category_spend_df.coalesce(1).write.mode("overwrite").parquet(f"/mnt/gold-layer/{date}/Top_Categories_by_Spending")